# Regression

In Machine Learning, regression is used when you have labelled data and your goal is to fit a continous output to input data. Note that for this definition of regression, something like logistic regression typically falls under classification, not regression.

This notebook covers creating a simple linear regression, some simple feature engineering (creating polynomial features, scaling features), regression model training and evaluation.

## Imports

Imports all of the libraries for the notebook

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_boston
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, ShuffleSplit
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.pipeline import make_pipeline

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import seaborn as sns

## Load in the data

This pulls in the Boston house-price dataset which is a commonly used dataset for regression. The attributes are:

- CRIM     per capita crime rate by town
- ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
- INDUS    proportion of non-retail business acres per town
- CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
- NOX      nitric oxides concentration (parts per 10 million)
- RM       average number of rooms per dwelling
- AGE      proportion of owner-occupied units built prior to 1940
- DIS      weighted distances to five Boston employment centres
- RAD      index of accessibility to radial highways
- TAX      full-value property-tax rate per \$10,000
- PTRATIO  pupil-teacher ratio by town
- B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
- LSTAT    % lower status of the population
- MEDV     Median value of owner-occupied homes in \$1000's

Typically, the first 13 are used as pottential variables to predict MEDV.

In addition, we split it into training and test for validation purposes.

In [3]:
data_boston = load_boston()
print("Shape:", data_boston.data.shape)

X_train, X_test, y_train, y_test = train_test_split(data_boston.data, data_boston.target, test_size=0.20)

Shape: (506, 13)


## Building Models


### A simple linear regression

This trains a one-off simple linear regression model.

In [9]:
# Define the model and hyper parameters
lin_model = LinearRegression(fit_intercept=True)
# Train the modl
lin_model.fit(X_train, y_train)
# Make predictions on the training data
y_pred = lin_model.predict(X_train)

# Score the model
train_r2 = lin_model.score(X_train, y_train)
test_r2 = lin_model.score(X_test, y_test)

print(("Training R^2: {:,.3f}").format(train_r2))
print(("Test R^2: {:,.3f}").format(test_r2))

print("A few predictions based on a simple model:")
simple_results = pd.DataFrame(X_train, columns=['CRIM','ZN','INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX','PTRATIO','B','LSTAT'])
simple_results['ACTUAL MDEV'] = y_train
simple_results['PREDICTD MDEV'] = y_pred
simple_results.head()

Training R^2: 0.732
Test R^2: 0.761
A few predictions based on a simple model:


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,ACTUAL MDEV,PREDICTD MDEV
0,9.32909,0.0,18.10,0.0,0.713,6.185,98.7,2.2616,24.0,666.0,20.2,396.90,18.13,14.1,17.674829
1,0.15038,0.0,25.65,0.0,0.581,5.856,97.0,1.9444,2.0,188.0,19.1,370.31,25.41,17.3,15.575196
2,0.13158,0.0,10.01,0.0,0.547,6.176,72.5,2.7301,6.0,432.0,17.8,393.30,12.04,21.2,23.190176
3,28.65580,0.0,18.10,0.0,0.597,5.155,100.0,1.5894,24.0,666.0,20.2,210.97,20.08,16.3,11.524100
4,0.09744,0.0,5.96,0.0,0.499,5.841,61.4,3.3779,5.0,279.0,19.2,377.56,11.41,20.0,22.553080


### A more complicated model

The previous code creates one model at a time. This isn't very efficient if you're trying to test a variety of hyper-parameters or model configurations to see what creates the "best" model. Here we do a few things a bit differently to simplify and automate training and evaluating models:

1. Define a pipeline that chains generating polynomial features with the linear regression model to create a polynomial regression model.
1. Use gridsearch to test each combination of hyper-parameters.
1. Loop through different linear regression models (that vary on the regularization used).

#### Note on regularization

In a linear regression model, complexity is associated with the number of non-zero coefficients and the magnitude of the coefficients. A more complicated model can fit more nuanced data, but is also more likely to overfit the training data. Regularization penalizes model complexity and serves a check on overfitting the data. A general form for L*p*-norm is:

$L_p(\mathbf{x}) = \sqrt[p]{\sum_i{\left | x_i^p\right |}}$


More concretely:

- l0 (uncommon): Number of non-zero coefficients.
- l1-norm (common): Sum of the absolute value of coefficients. Regularizing with this is commonly referred to as lasso. 
- l2-norm (common): Square root of the sum of squares of coefficients. Regularizing with this is commonly referred to as ridge.
- l-infinity-norm (uncommon): The maximum of the coefficients.

In [9]:
def PolynomialRegression(degree=2, linear_model=LinearRegression(), **kwargs):
    return make_pipeline(PolynomialFeatures(degree), linear_model(**kwargs))

def fit_model(X, y):
    # Create cross-validation sets from the training data
    cv_sets = ShuffleSplit(n_splits=10, test_size = 0.20, random_state = 0)
 
    #Creating a dictionary for the parameter degree with a range from 1 to 10 and fit_intercept of true or false
    params = {'polynomialfeatures__degree': [1,2,3,4]}

    best_models = []
    for m in [LinearRegression, Lasso, Ridge]:
        grid = GridSearchCV(PolynomialRegression(linear_model=m), param_grid = params, 
                         cv=cv_sets, 
                         scoring='r2',
                         return_train_score=True)
        grid = grid.fit(X, y)
        cv_results = pd.DataFrame(grid.cv_results_)
        print(m)
        print(cv_results[['params', 'mean_fit_time', 'mean_score_time', 'mean_test_score']].sort_values(by='mean_test_score', ascending=False))
        best_models.append(grid.best_estimator_)
    
    return best_models

#Fitting the training data using grid search
models = fit_model(X_train, y_train)

<class 'sklearn.linear_model.base.LinearRegression'>
                              params  mean_fit_time  mean_score_time  \
1  {'polynomialfeatures__degree': 2}       0.007115         0.002431   
0  {'polynomialfeatures__degree': 1}       0.002324         0.000949   
3  {'polynomialfeatures__degree': 4}       0.135308         0.048760   
2  {'polynomialfeatures__degree': 3}       0.055854         0.010824   

   mean_test_score  
1         0.732345  
0         0.717943  
3     -1441.482409  
2     -1597.457245  
<class 'sklearn.linear_model.coordinate_descent.Lasso'>
                              params  mean_fit_time  mean_score_time  \
1  {'polynomialfeatures__degree': 2}       0.037231         0.003431   
0  {'polynomialfeatures__degree': 1}       0.001051         0.000791   
2  {'polynomialfeatures__degree': 3}       0.199968         0.011112   
3  {'polynomialfeatures__degree': 4}       0.881153         0.033844   

   mean_test_score  
1         0.828389  
0         0.669705  
2

C:\Users\CobyLU\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:152: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.686867e-19
  overwrite_a=False)
C:\Users\CobyLU\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:152: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.813703e-19
  overwrite_a=False)


<class 'sklearn.linear_model.ridge.Ridge'>
                              params  mean_fit_time  mean_score_time  \
1  {'polynomialfeatures__degree': 2}       0.003511         0.002150   
0  {'polynomialfeatures__degree': 1}       0.001305         0.000481   
2  {'polynomialfeatures__degree': 3}       0.040669         0.010342   
3  {'polynomialfeatures__degree': 4}       0.095353         0.044213   

   mean_test_score  
1         0.821751  
0         0.714930  
2        -5.665710  
3      -180.263430  


#### Model coefficients with regularization

Here's a bit of code to look at the coefficients produced by lasso and ridge regularization to illustrate that is is, in fact, reducing the magnitude of the coefficients.

In [29]:
print(models[0].named_steps['linearregression'].coef_.mean())
print(models[1].named_steps['lasso'].coef_)
print(models[2].named_steps['ridge'].coef_.mean())

25.452603718075174
[ 0.         -0.          0.         -0.          0.         -0.
  0.         -0.          0.         -0.         -2.42932061 -0.
  0.         -7.8146962 ]
-0.01674592237683825


### Feature Engineering

Our attributes can exist on radically different scales. i.e. number of rooms might on the order of 1-10 and the of the house could be on the order of 1-100. Being on the extreme one of the ranges could be useful to our regression model, but the regularization on the coefficients biases towards giving more importance to attributes that typically have large values i.e. biasing towards using age over number of rooms.

Here we apply a min-max scaler to scale the different attributes. The min-max scaler scales each attribute so the minimum is 0 and the maximum is 1. A reasonable alternative could be to normalize the variables which sets the mean to 0 and the standard deviation to 1. Variables may still skew if the data isn't normal.

In [14]:
features_transformed = pd.DataFrame(data=data_boston.data, columns=data_boston.feature_names)

# Applying scaling using MinMaxScaler
scaler = MinMaxScaler()
features_transformed[data_boston.feature_names] = scaler.fit_transform(features_transformed)
print(features_transformed.head())

       CRIM    ZN     INDUS  CHAS       NOX        RM       AGE       DIS  \
0  0.000000  0.18  0.067815   0.0  0.314815  0.577505  0.641607  0.269203   
1  0.000236  0.00  0.242302   0.0  0.172840  0.547998  0.782698  0.348962   
2  0.000236  0.00  0.242302   0.0  0.172840  0.694386  0.599382  0.348962   
3  0.000293  0.00  0.063050   0.0  0.150206  0.658555  0.441813  0.448545   
4  0.000705  0.00  0.063050   0.0  0.150206  0.687105  0.528321  0.448545   

        RAD       TAX   PTRATIO         B     LSTAT  
0  0.000000  0.208015  0.287234  1.000000  0.089680  
1  0.043478  0.104962  0.553191  1.000000  0.204470  
2  0.043478  0.104962  0.553191  0.989737  0.063466  
3  0.086957  0.066794  0.648936  0.994276  0.033389  
4  0.086957  0.066794  0.648936  1.000000  0.099338  


#### Feature Engineering impact on model perforamance

Scaling and normalizing variables **should** make better models. In practice, it doesn't always work. When conducting an initial investigation, with the goal of getting the "most accurate" model, a common practice is to build simple models first. The simple model doesn't need things like feature engineering or scaling, but it provids a baseline to compare against more "correct" or advanced models' performance. In an agile sense, it can also act as a potential MVP (minimum viable product) model.

In [15]:
X_train_scale, X_test_scale, y_train_scale, y_test_scale = train_test_split(features_transformed, 
                    data_boston.target, test_size = 0.2, random_state = 0)
models = fit_model(X_train_scale, y_train_scale)

<class 'sklearn.linear_model.base.LinearRegression'>
                              params  mean_fit_time  mean_score_time  \
1  {'polynomialfeatures__degree': 2}       0.006915         0.002533   
0  {'polynomialfeatures__degree': 1}       0.004193         0.001001   
3  {'polynomialfeatures__degree': 4}       0.137997         0.049908   
2  {'polynomialfeatures__degree': 3}       0.055528         0.013584   

   mean_test_score  
1         0.751177  
0         0.725487  
3       -10.466120  
2       -22.506637  
<class 'sklearn.linear_model.coordinate_descent.Lasso'>
                              params  mean_fit_time  mean_score_time  \
0  {'polynomialfeatures__degree': 1}       0.002511         0.000873   
2  {'polynomialfeatures__degree': 3}       0.021671         0.010345   
3  {'polynomialfeatures__degree': 4}       0.106638         0.041869   
1  {'polynomialfeatures__degree': 2}       0.005462         0.002647   

   mean_test_score  
0         0.292238  
2         0.290059  
3

#### Putting it together

Here's a block of code that has the scaling and model training all together.

In [12]:
features_transformed = pd.DataFrame(data=data_boston.data, columns=data_boston.feature_names)

scaler = StandardScaler()
features_transformed[data_boston.feature_names] = scaler.fit_transform(features_transformed)
print(features_transformed.head())

X_train_scale, X_test_scale, y_train_scale, y_test_scale = train_test_split(features_transformed, 
                    data_boston.target, test_size = 0.2, random_state = 0)
models = fit_model(X_train_scale, y_train_scale)

       CRIM        ZN     INDUS      CHAS       NOX        RM       AGE  \
0 -0.417713  0.284830 -1.287909 -0.272599 -0.144217  0.413672 -0.120013   
1 -0.415269 -0.487722 -0.593381 -0.272599 -0.740262  0.194274  0.367166   
2 -0.415272 -0.487722 -0.593381 -0.272599 -0.740262  1.282714 -0.265812   
3 -0.414680 -0.487722 -1.306878 -0.272599 -0.835284  1.016303 -0.809889   
4 -0.410409 -0.487722 -1.306878 -0.272599 -0.835284  1.228577 -0.511180   

        DIS       RAD       TAX   PTRATIO         B     LSTAT  
0  0.140214 -0.982843 -0.666608 -1.459000  0.441052 -1.075562  
1  0.557160 -0.867883 -0.987329 -0.303094  0.441052 -0.492439  
2  0.557160 -0.867883 -0.987329 -0.303094  0.396427 -1.208727  
3  1.077737 -0.752922 -1.106115  0.113032  0.416163 -1.361517  
4  1.077737 -0.752922 -1.106115  0.113032  0.441052 -1.026501  
<class 'sklearn.linear_model.base.LinearRegression'>
                              params  mean_fit_time  mean_score_time  \
1  {'polynomialfeatures__degree': 2}    